In [1]:
import os

In [2]:
%pwd

'/home/surajkarki/Documents/My-Workspace/My-Work/Final-Year-Project/MediLeaf_AI/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/surajkarki/Documents/My-Workspace/My-Work/Final-Year-Project/MediLeaf_AI'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from MediLeaf_AI.constants import *
from MediLeaf_AI.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config     

In [8]:
import os
import urllib.request as request
import zipfile
from MediLeaf_AI import logger
from MediLeaf_AI.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-24 11:14:27,361: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-06-24 11:14:27,373: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-24 11:14:27,375: INFO: common: created directory at: artifacts]
[2023-06-24 11:14:27,378: INFO: common: created directory at: artifacts/data_ingestion]


[2023-06-24 11:15:14,644: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Date: Sat, 24 Jun 2023 05:29:28 GMT
Content-Type: application/zip
Content-Length: 456435396
Connection: close
Cache-Control: max-age=60, public, must-revalidate, stale-while-revalidate=60, stale-if-error=300, s-maxage=60
Content-Disposition: attachment; filename="dataset_v1.zip"; filename*=UTF-8''dataset_v1.zip
Content-Security-Policy: base-uri 'self'; child-src https://www.google.com/recaptcha/ https://www.recaptcha.net/ https://content.googleapis.com https://content-compute.googleapis.com https://content-cloudbilling.googleapis.com https://content-cloudresourcemanager.googleapis.com https://www.googletagmanager.com/ns.html https://*.zuora.com/apps/PublicHostedPageLite.do https://gitlab.com/admin/ https://gitlab.com/assets/ https://gitlab.com/-/speedscope/index.html https://gitlab.com/-/sandbox/ https://gitlab.com/assets/ blob: data:; connect-src 'self' https://gitlab.com wss: